In [1]:
from sqlalchemy import create_engine
import random
import pandas as pd
import numpy as np
import names
import string

### Ufuncs

def _clamp(x):
    return max(0, min(x, 255))


def col_gen():
    return "#{0:02x}{1:02x}{2:02x}".format(
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
    )

### Download few sample standard datasets

df = pd.read_csv("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/4_ThreeNum.csv")
disk_engine = create_engine('sqlite:///charts_db.db')
df.to_sql('gdp_data', disk_engine, if_exists='append')
df = pd.read_csv("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/weather.csv")
disk_engine = create_engine('sqlite:///charts_db.db')
df.to_sql('weather_data', disk_engine, if_exists='append')
df = pd.read_csv("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/6_SeveralNum.csv")
df.rename(columns = {"Unnamed: 0":"label"}, inplace = True)
disk_engine = create_engine('sqlite:///charts_db.db')
df.to_sql('mtcars_data', disk_engine, if_exists='append')
pd.read_csv("https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/data_correlogram.csv").rename(columns = {"Unnamed: 0":"label"}).round(4).to_sql('correlogram_data_table', disk_engine, if_exists='append')

11

## __Donut chart:__

* Two tables: data and fill
* In data table it will have two columns: label and value
* In fill table it will have two columns: label and color
* As next step we need to have a drop down menu which will filter the data which has to be shown in the chart and in the backend we will send a list of key value pair where value will be of the same format as it is for a single selection of the drop down menu.

### __Expected output:__

```
[{'label': 'Michael Tacadina', 'value': 24, 'fill': '#a72b31'},
 {'label': 'John Thomas', 'value': 22, 'fill': '#fdb134'},
 {'label': 'Carrie Carter', 'value': 81, 'fill': '#395f5b'},
 {'label': 'Catherine Henry', 'value': 33, 'fill': '#2cdcf3'},
 {'label': 'Stephanie Horton', 'value': 49, 'fill': '#8df85c'},
 {'label': 'Reggie Miller', 'value': 23, 'fill': '#37c19f'},
 {'label': 'Bernard Bushman', 'value': 6, 'fill': '#74ecd7'},
 {'label': 'Kenneth Rubio', 'value': 98, 'fill': '#e1d1c1'},
 {'label': 'Idella Corda', 'value': 35, 'fill': '#ca7cff'},
 {'label': 'Kay Kaye', 'value': 22, 'fill': '#79d2b6'}]
```

* __Reference:__ https://codesandbox.io/s/6hr9o

In [2]:
### Generate data

label_len = 10
label = [names.get_full_name() for _ in range(label_len)]
value = [random.randint(1, 100) for _ in range(label_len)]
fill = [col_gen() for _ in range(label_len)]
donut_df = pd.DataFrame({'label': label, 'value': value})
label_fill_df = pd.DataFrame({'label': label, 'fill': fill})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
donut_df.to_sql('donut_data_table', disk_engine, if_exists='append')
label_fill_df.to_sql('donut_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM donut_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM donut_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = ((data_df.set_index('label')).join(fill_df.set_index('label'))).reset_index().to_dict('records')
response_payload

[{'label': 'Clifford Rodriguez', 'value': 72, 'fill': '#0e8dc7'},
 {'label': 'Bruce Leth', 'value': 50, 'fill': '#8047de'},
 {'label': 'Sherri Trautman', 'value': 54, 'fill': '#645566'},
 {'label': 'Ramiro Stewart', 'value': 89, 'fill': '#8c3d7c'},
 {'label': 'James Finchum', 'value': 37, 'fill': '#a7ccf6'},
 {'label': 'Barbara Miller', 'value': 41, 'fill': '#c3fb46'},
 {'label': 'Cindy Varin', 'value': 75, 'fill': '#754b44'},
 {'label': 'Rosalie Biron', 'value': 2, 'fill': '#1b6617'},
 {'label': 'Myrtle Smiley', 'value': 20, 'fill': '#b840db'},
 {'label': 'Steve Kelly', 'value': 45, 'fill': '#e7b8c5'}]

# __Stacked Bar Chart__

* `label` : x axis
* `q, y,x`: values to be plotted in y axis
* `fill`: colors of `q, y, x` to be plotted in chart


### __Expected output format:__

```
{'data': [{'label': 'Jordan Yates', 'y': 58.19, 'q': 87.72, 's': 13.3},
  {'label': 'Donald Mandich', 'y': 51.34, 'q': 29.0, 's': 85.66},
  {'label': 'Marie Cox', 'y': 51.56, 'q': 68.82, 's': 56.37},
  {'label': 'Glenn Davis', 'y': 37.87, 'q': 37.81, 's': 4.66},
  {'label': 'Judith Mouret', 'y': 79.04, 'q': 2.17, 's': 62.1},
  {'label': 'Gregoria Schupp', 'y': 47.83, 'q': 35.85, 's': 28.11}],
 'fill': [{'label': 'y', 'fill': '#d14aac'},
  {'label': 'q', 'fill': '#2dff71'},
  {'label': 's', 'fill': '#bd825e'}]}
```


* __Reference:__ https://www.geeksforgeeks.org/create-a-stacked-bar-chart-using-recharts-in-reactjs/

In [3]:
### Generate data

n_label = 6
n_var = 3
label = [names.get_full_name() for _ in range(n_label)]
var_collector = []
data_collector = []
cols_collector = []

while len(set(var_collector))< n_var:
    var_collector.append(random.choice(string.ascii_lowercase))
    data_collector.append(np.random.uniform(1,100,n_label))
    cols_collector.append(col_gen())

data_df = pd.DataFrame(data_collector, index =var_collector, columns = label).T.reset_index().rename(columns = {"index":"label"}).round(2)
fill_df = pd.DataFrame({'label': var_collector, 'fill': cols_collector})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
data_df.to_sql('stackedchart_data_table', disk_engine, if_exists='append')
fill_df.to_sql('stackedchart_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM stackedchart_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM stackedchart_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = {"data":data_df.to_dict('records'),"fill":fill_df.to_dict('records')}
response_payload

{'data': [{'label': 'Charles Soule', 'l': 72.53, 'r': 19.79, 'i': 58.17},
  {'label': 'Danny Goan', 'l': 89.91, 'r': 45.46, 'i': 35.69},
  {'label': 'Wayne Miles', 'l': 79.96, 'r': 8.97, 'i': 46.92},
  {'label': 'Tanya David', 'l': 85.69, 'r': 56.69, 'i': 83.47},
  {'label': 'Edward Adkinson', 'l': 12.01, 'r': 60.49, 'i': 27.21},
  {'label': 'Bette Bailey', 'l': 92.06, 'r': 96.48, 'i': 35.76}],
 'fill': [{'label': 'l', 'fill': '#b01a0f'},
  {'label': 'r', 'fill': '#69ab7f'},
  {'label': 'i', 'fill': '#d837d6'}]}

# Line chart

* `label` values to be shown in x axis
* `q, j. o` are values to be shown in y axis. this number may vary
* `fill` the color code of the q, j and o has to be populated from `fill` key

### __Expected output format:__


```
{'data': [{'label': 'Elsie Bahr', 'q': 4.06, 'j': 80.86, 'o': 53.18},
  {'label': 'Barbara Brown', 'q': 14.81, 'j': 2.42, 'o': 13.32},
  {'label': 'Kyle Garza', 'q': 29.38, 'j': 84.08, 'o': 12.12},
  {'label': 'John Miller', 'q': 70.11, 'j': 53.58, 'o': 43.44},
  {'label': 'Kermit Collison', 'q': 48.75, 'j': 64.24, 'o': 35.52},
  {'label': 'Philip Oliveira', 'q': 98.84, 'j': 26.04, 'o': 69.33}],
 'fill': [{'label': 'q', 'fill': '#b73fba'},
  {'label': 'j', 'fill': '#51a3cc'},
  {'label': 'o', 'fill': '#7f9356'}]}

```
* __Reference:__ https://www.geeksforgeeks.org/create-a-line-chart-using-recharts-in-reactjs/

In [4]:
### Generate data
n_label = 6
n_var = 3
label = [names.get_full_name() for _ in range(n_label)]
var_collector = []
data_collector = []
cols_collector = []

while len(set(var_collector))< n_var:
    var_collector.append(random.choice(string.ascii_lowercase))
    data_collector.append(np.random.uniform(1,100,n_label))
    cols_collector.append(col_gen())

data_df = pd.DataFrame(data_collector, index =var_collector, columns = label).T.reset_index().rename(columns = {"index":"label"}).round(2)
fill_df = pd.DataFrame({'label': var_collector, 'fill': cols_collector})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
data_df.to_sql('linechart_data_table', disk_engine, if_exists='append')
fill_df.to_sql('linechart_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM linechart_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM linechart_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = {"data":data_df.to_dict('records'),"fill":fill_df.to_dict('records')}
response_payload

{'data': [{'label': 'Ruby Lichtenberg', 'i': 27.89, 'n': 49.48, 'o': 36.67},
  {'label': 'Stanley Crawford', 'i': 79.99, 'n': 79.35, 'o': 69.11},
  {'label': 'Guadalupe Green', 'i': 8.21, 'n': 60.06, 'o': 66.48},
  {'label': 'Jeremy Meade', 'i': 5.27, 'n': 1.95, 'o': 41.57},
  {'label': 'Vincent Barton', 'i': 67.82, 'n': 60.3, 'o': 63.83},
  {'label': 'Stephanie Langton', 'i': 3.36, 'n': 68.51, 'o': 55.87}],
 'fill': [{'label': 'i', 'fill': '#3640d5'},
  {'label': 'i', 'fill': '#9f1a25'},
  {'label': 'n', 'fill': '#2625ee'},
  {'label': 'o', 'fill': '#c7b8a8'}]}

# Area chart

* __Reference:__ https://www.geeksforgeeks.org/create-a-stacked-area-chart-using-recharts-in-reactjs/

In [5]:
### Generate data
n_label = 6
n_var = 3
label = [names.get_full_name() for _ in range(n_label)]
var_collector = []
data_collector = []
cols_collector = []

while len(set(var_collector))< n_var:
    var_collector.append(random.choice(string.ascii_lowercase))
    data_collector.append(np.random.uniform(1,100,n_label))
    cols_collector.append(col_gen())

data_df = pd.DataFrame(data_collector, index =var_collector, columns = label).T.reset_index().rename(columns = {"index":"label"}).round(2)
fill_df = pd.DataFrame({'label': var_collector, 'fill': cols_collector})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
data_df.to_sql('areachart_data_table', disk_engine, if_exists='append')
fill_df.to_sql('areachart_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM areachart_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM areachart_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = {"data":data_df.to_dict('records'),"fill":fill_df.to_dict('records')}
response_payload

{'data': [{'label': 'Sandra Olds', 'e': 84.48, 'b': 6.28, 'j': 46.22},
  {'label': 'Deborah Moore', 'e': 92.23, 'b': 23.45, 'j': 37.39},
  {'label': 'John Huston', 'e': 73.77, 'b': 58.23, 'j': 69.09},
  {'label': 'Linda Woods', 'e': 24.29, 'b': 67.61, 'j': 71.83},
  {'label': 'Joy Davis', 'e': 19.58, 'b': 1.49, 'j': 23.05},
  {'label': 'Sophia Thompson', 'e': 34.83, 'b': 69.56, 'j': 60.53}],
 'fill': [{'label': 'e', 'fill': '#24aec1'},
  {'label': 'b', 'fill': '#5dea4a'},
  {'label': 'j', 'fill': '#af591c'}]}

# Scatter plot

* __Reference:__ https://www.geeksforgeeks.org/create-a-scatter-chart-using-recharts-in-reactjs/

In [6]:
### Generate data
n_label = 6
n_var = 2
var_collector = []
data_collector = []
cols_collector = []

while len(set(var_collector)) < n_var:
    var_collector.append(random.choice(string.ascii_lowercase))
    data_collector.append(np.random.uniform(1,100,n_label))
    cols_collector.append(col_gen())


data_df = pd.DataFrame(data_collector, index =var_collector).T.round(2)
fill_df = pd.DataFrame({'label': var_collector, 'fill': cols_collector})


# ### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
data_df.to_sql('scatterplot_data_table', disk_engine, if_exists='append')
fill_df.to_sql('scatterplot_fill_table', disk_engine, if_exists='append')

# ### Get data

data_df = pd.read_sql_query('SELECT * FROM scatterplot_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM scatterplot_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


# ### Parse data for chart

response_payload = {"data":data_df.to_dict('records'),"fill":dict(zip(fill_df['label'].tolist(), fill_df['fill'].tolist()))}
response_payload

{'data': [{'k': 94.16, 'y': 88.2},
  {'k': 26.79, 'y': 85.36},
  {'k': 11.97, 'y': 94.24},
  {'k': 48.81, 'y': 8.07},
  {'k': 73.73, 'y': 3.75},
  {'k': 31.09, 'y': 68.4}],
 'fill': {'k': '#de232c', 'y': '#6c1c0f'}}

# Bar Chart


* __Reference:__ https://www.geeksforgeeks.org/create-a-bar-chart-using-recharts-in-reactjs/

In [7]:
### Generate data

label_len = 10
label = [names.get_full_name() for _ in range(label_len)]
value = [random.randint(1, 100) for _ in range(label_len)]
fill = [col_gen() for _ in range(label_len)]
donut_df = pd.DataFrame({'label': label, 'value': value})
label_fill_df = pd.DataFrame({'label': label, 'fill': fill})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
donut_df.to_sql('barchart_data_table', disk_engine, if_exists='append')
label_fill_df.to_sql('barchart_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM barchart_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM barchart_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = ((data_df.set_index('label')).join(fill_df.set_index('label'))).reset_index().to_dict('records')
response_payload

[{'label': 'Minh Sauceda', 'value': 38, 'fill': '#245aa0'},
 {'label': 'Todd Beaucage', 'value': 40, 'fill': '#31b342'},
 {'label': 'Ronald Blenden', 'value': 97, 'fill': '#bf4872'},
 {'label': 'Martha Roby', 'value': 18, 'fill': '#6151f7'},
 {'label': 'Nicholas Rottman', 'value': 68, 'fill': '#c8564c'},
 {'label': 'Susan High', 'value': 45, 'fill': '#0a0d1b'},
 {'label': 'Eric Ballard', 'value': 83, 'fill': '#5f3bb8'},
 {'label': 'Johnny Hernandez', 'value': 36, 'fill': '#4e68e5'},
 {'label': 'Randy Surratt', 'value': 25, 'fill': '#c8746a'},
 {'label': 'Ruth Greene', 'value': 79, 'fill': '#220bb2'}]

# Bubble chart


* `gdpPercap`: x axis
* `lifeExp`: y axis
* `pop`: size of bubble
* `continent`: colour of bubble


* __Reference:__ https://d3-graph-gallery.com/graph/bubble_basic.html

In [8]:
### Read data from DB

df = pd.read_sql_query('SELECT * FROM gdp_data',disk_engine)
df = df.drop(['index'], axis = 1)
df = df.rename(columns = {"gdpPercap":"x","lifeExp":"y","pop":"z","country":'label'})
colour_df = df[['continent']].drop_duplicates()
colour_df['fill'] = [col_gen() for _ in range(len(colour_df))]
disk_engine = create_engine('sqlite:///charts_db.db')
colour_df.to_sql('gdp_data_fill', disk_engine, if_exists='append')

### Parse data for chart

response_payload = {"data":df.to_dict("records"),"fill":colour_df.to_dict("records")}
response_payload

{'data': [{'label': 'Afghanistan',
   'continent': 'Asia',
   'y': 43.828,
   'z': 31889923,
   'x': 974.5803384},
  {'label': 'Albania',
   'continent': 'Europe',
   'y': 76.423,
   'z': 3600523,
   'x': 5937.029526},
  {'label': 'Algeria',
   'continent': 'Africa',
   'y': 72.301,
   'z': 33333216,
   'x': 6223.367465},
  {'label': 'Angola',
   'continent': 'Africa',
   'y': 42.731,
   'z': 12420476,
   'x': 4797.231267},
  {'label': 'Argentina',
   'continent': 'Americas',
   'y': 75.32,
   'z': 40301927,
   'x': 12779.37964},
  {'label': 'Australia',
   'continent': 'Oceania',
   'y': 81.235,
   'z': 20434176,
   'x': 34435.36744},
  {'label': 'Austria',
   'continent': 'Europe',
   'y': 79.829,
   'z': 8199783,
   'x': 36126.4927},
  {'label': 'Bahrain',
   'continent': 'Asia',
   'y': 75.635,
   'z': 708573,
   'x': 29796.04834},
  {'label': 'Bangladesh',
   'continent': 'Asia',
   'y': 64.062,
   'z': 150448339,
   'x': 1391.253792},
  {'label': 'Belgium',
   'continent': 'Europ

# Correlogram

* __Reference:__ https://d3-graph-gallery.com/graph/correlogram_basic.html

In [9]:
### Read data from DB

df = pd.read_sql_query('SELECT * FROM correlogram_data_table',disk_engine)


### Parse data for chart

df = df.drop(['index'], axis = 1)
df = df.set_index('label')
df.index.name = None
df_dict = df.to_dict("split")
data_collector = []
for x in df_dict["index"]:
    for y in df_dict["columns"]:
        data_collector.append(
            {"x": x, "y": y, "value": float(df.loc[x, y].round(4))}
        )
response_payload = {"data":data_collector,"fill":None}
response_payload

{'data': [{'x': 'mpg', 'y': 'mpg', 'value': 1.0},
  {'x': 'mpg', 'y': 'cyl', 'value': -0.8522},
  {'x': 'mpg', 'y': 'disp', 'value': -0.8476},
  {'x': 'mpg', 'y': 'hp', 'value': -0.7762},
  {'x': 'mpg', 'y': 'drat', 'value': 0.6812},
  {'x': 'mpg', 'y': 'wt', 'value': -0.8677},
  {'x': 'mpg', 'y': 'qsec', 'value': 0.4187},
  {'x': 'mpg', 'y': 'vs', 'value': 0.664},
  {'x': 'mpg', 'y': 'am', 'value': 0.5998},
  {'x': 'mpg', 'y': 'gear', 'value': 0.4803},
  {'x': 'mpg', 'y': 'carb', 'value': -0.5509},
  {'x': 'cyl', 'y': 'mpg', 'value': -0.8522},
  {'x': 'cyl', 'y': 'cyl', 'value': 1.0},
  {'x': 'cyl', 'y': 'disp', 'value': 0.902},
  {'x': 'cyl', 'y': 'hp', 'value': 0.8324},
  {'x': 'cyl', 'y': 'drat', 'value': -0.6999},
  {'x': 'cyl', 'y': 'wt', 'value': 0.7825},
  {'x': 'cyl', 'y': 'qsec', 'value': -0.5912},
  {'x': 'cyl', 'y': 'vs', 'value': -0.8108},
  {'x': 'cyl', 'y': 'am', 'value': -0.5226},
  {'x': 'cyl', 'y': 'gear', 'value': -0.4927},
  {'x': 'cyl', 'y': 'carb', 'value': 0.527}

# Histogram


* __Reference:__ https://codesandbox.io/s/new-field-u5oze

In [10]:
### Generate data

step=50
s = pd.Series(np.random.uniform(0, 1000, size=1000)) # This is the data
bin_range = np.arange(-200, 1000 + step, step)
out, bins = pd.cut(
    s, bins=bin_range, include_lowest=True, right=False, retbins=True
)
df = (
    out.value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "name", 0: "freq"})
)
x_col = []
for x in df["name"]:
    x_col.append(x.mid)
df["name"] = df["name"].astype(str)
df["name"] = df["name"].str.replace("\[", "", regex=True)
df["name"] = df["name"].str.replace("\)", "", regex=True)
df["name"] = df["name"].str.replace("\,", " -", regex=True)
df["value"] = x_col
df = df.rename(columns={"name": "label"})

### Parse data for chart



response_payload = {"data":df.to_dict("records"),"fill":col_gen()}
response_payload

{'data': [{'label': '100 - 150', 'freq': 64, 'value': 125.0},
  {'label': '650 - 700', 'freq': 63, 'value': 675.0},
  {'label': '200 - 250', 'freq': 56, 'value': 225.0},
  {'label': '850 - 900', 'freq': 54, 'value': 875.0},
  {'label': '550 - 600', 'freq': 54, 'value': 575.0},
  {'label': '50 - 100', 'freq': 54, 'value': 75.0},
  {'label': '750 - 800', 'freq': 52, 'value': 775.0},
  {'label': '600 - 650', 'freq': 52, 'value': 625.0},
  {'label': '450 - 500', 'freq': 52, 'value': 475.0},
  {'label': '700 - 750', 'freq': 50, 'value': 725.0},
  {'label': '250 - 300', 'freq': 48, 'value': 275.0},
  {'label': '150 - 200', 'freq': 48, 'value': 175.0},
  {'label': '0 - 50', 'freq': 47, 'value': 25.0},
  {'label': '350 - 400', 'freq': 46, 'value': 375.0},
  {'label': '300 - 350', 'freq': 46, 'value': 325.0},
  {'label': '500 - 550', 'freq': 46, 'value': 525.0},
  {'label': '400 - 450', 'freq': 46, 'value': 425.0},
  {'label': '800 - 850', 'freq': 44, 'value': 825.0},
  {'label': '950 - 1000', 

# Waterfall chart


* __Reference:__ https://codesandbox.io/s/new-field-u5oze

In [11]:
def random_dates(
    start=pd.to_datetime("2015-01-01"),
    end=pd.to_datetime("2018-01-01"),
    unit="D",
    seed=None,
):
    n = random.randint(4, 12)
    if not seed:  # from piR's answer
        np.random.seed(0)
    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.rand(n) * ndays, unit=unit) + start


def _waterfall_payload_generator():
    dates_list = (
        pd.Series(random_dates().sort_values())
        .dt.to_period("M")
        .drop_duplicates()
        .astype(str)
        .tolist()
    )
    old_list = [{"uv": float(np.random.uniform(2700, 4500, 1)), "pv": 0}]
    for index in range(1, len(dates_list)):
        old_list.append(
            {
                "uv": old_list[index - 1]["uv"] + old_list[index - 1]["pv"],
                "pv": float(np.random.uniform(-500, 500, 1)),
            }
        )
    old_list[len(dates_list) - 1]["pv"] = 0
    df = pd.DataFrame(old_list)
    df["date"] = dates_list
    df = df.round(2)
    df["date"] = df["date"].astype(str)
    df = df.rename(columns = {'date':'label'})
    df.loc[df['label'] ==df['label'].max(),'label'] = 'total'
    return df.to_dict("records")


response_payload = {"data":_waterfall_payload_generator(),"fill":None}
response_payload

{'data': [{'uv': 3652.01, 'pv': 0.0, 'label': '2016-02'},
  {'uv': 3652.01, 'pv': 68.04, 'label': '2016-04'},
  {'uv': 3720.06, 'pv': 425.6, 'label': '2016-08'},
  {'uv': 4145.65, 'pv': -428.96, 'label': '2016-10'},
  {'uv': 3716.69, 'pv': -412.87, 'label': '2016-12'},
  {'uv': 3303.82, 'pv': -479.78, 'label': '2017-02'},
  {'uv': 2824.04, 'pv': 332.62, 'label': '2017-05'},
  {'uv': 3156.66, 'pv': 278.16, 'label': '2017-09'},
  {'uv': 3434.81, 'pv': 0.0, 'label': 'total'}],
 'fill': None}